In [108]:
import pandas as pd;

In [131]:
sheets = ["rulespergoal8.csv" , "rulespergoal5.csv" , "rulespergoal3.csv" , 
          "goals100beliefs1000.csv" , "goals50beliefs1000.csv", "goals100beliefs500.csv",
         "dynamics01.csv" , "dynamics02.csv"];
RULES8 = 0 ; RULES5 = 1; RULES3 = 2; 
G100B1000 = 3 , G50B1000 = 4 ; G100B500 = 
sheet = pd.read_csv(sheets[1] , sep =";", decimal =",")

In [132]:
tipoMemoria = [ "DEFAULT" , "STORING_RETRIEVING"  , "FORGETTING"];
DEFAULT = 0; STORING = 1; FORGETTING = 2;
tipoRaciocinio = ["SOAREngine" , "BDIEngine" , "BBGPEngine"];
SOAR = 0; BDI = 1; BBGP = 2;
tipoModeloRelevancia = ["DefaultRelevanceModel" , "BBGPIntegratedRelevanceModel" ,
                         "BDIIntegratedRelevanceModel" , "SOARIntegratedRelevanceModel "];
DEFAULT_MODEL = 0; BBGP_MODEL = 1; BDI_MODEL = 2; SOAR_MODEL = 3;

In [133]:
def filtro(tipoMemoria , tipoRaciocinio, tipoModeloRelevancia):
    return sheet.loc[(a['tipoDeMemoria'].str.contains(tipoMemoria)) 
                   & (a['modeloDeRelevancia'].str.contains(tipoModeloRelevancia))
                     & (a['tipoDeRaciocinio'].str.contains(tipoRaciocinio))]

In [134]:
filtrado = filtro(tipoMemoria[DEFAULT],tipoRaciocinio[SOAR], tipoModeloRelevancia[DEFAULT_MODEL]);

In [135]:
filtrado['eficienciaMemoria'].describe()

count    64.000000
mean      0.002100
std       0.002607
min       0.000000
25%       0.000000
50%       0.000766
75%       0.004854
max       0.007849
Name: eficienciaMemoria, dtype: float64

In [136]:
filtrado['eficienciaCpu'].describe()

count    64.000000
mean      0.079044
std       0.079779
min       0.000000
25%       0.000000
50%       0.081206
75%       0.126738
max       0.345461
Name: eficienciaCpu, dtype: float64